<a href="https://www.kaggle.com/code/khushboos/titanic?scriptVersionId=117920118" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split, cross_val_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
submission_data = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
# print(f"{gender_data.head(5)}\n\n Length:{len(gender_data)}\n\n{gender_data.isnull().sum()}")

In [3]:
# features = ["PassengerId", "Sex", "Age", "SibSp", "Parch","Fare", "Embarked"]
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
x_train = train_data.drop(['Survived', 'Pclass', 'Name', 'Ticket', 'Cabin'], axis= 1)
x_test = test_data.drop(['Pclass','Name','Ticket','Cabin'], axis=1)

y_train = train_data['Survived']

x_train['Sex'] = x_train['Sex'].map({'male':int(0), 'female':int(1)})
x_train['Embarked'] = x_train['Embarked'].map({'S':int(0), 'C':int(1), 'Q':int(2)})

x_test['Sex'] = x_test['Sex'].map({'male':int(0), 'female':int(1)})
x_test['Embarked'] = x_test['Embarked'].map({'S':int(0), 'C':int(1), 'Q':int(2)})


In [5]:
print(f"{x_train[x_train.isna().any(axis=1)]}\n\n\n{x_test[x_test.isna().any(axis=1)]}")
# df1

     PassengerId  Sex  Age  SibSp  Parch     Fare  Embarked
5              6    0  NaN      0      0   8.4583       2.0
17            18    0  NaN      0      0  13.0000       0.0
19            20    1  NaN      0      0   7.2250       1.0
26            27    0  NaN      0      0   7.2250       1.0
28            29    1  NaN      0      0   7.8792       2.0
..           ...  ...  ...    ...    ...      ...       ...
859          860    0  NaN      0      0   7.2292       1.0
863          864    1  NaN      8      2  69.5500       0.0
868          869    0  NaN      0      0   9.5000       0.0
878          879    0  NaN      0      0   7.8958       0.0
888          889    1  NaN      1      2  23.4500       0.0

[179 rows x 7 columns]


     PassengerId  Sex  Age  SibSp  Parch     Fare  Embarked
10           902    0  NaN      0      0   7.8958         0
22           914    1  NaN      0      0  31.6833         0
29           921    0  NaN      2      0  21.6792         1
33           9

In [6]:
x_train["Age"].fillna(value=x_train['Age'].mean(), inplace=True)
x_train["Embarked"].fillna(value=x_train['Embarked'].mean(), inplace=True)

x_test["Age"].fillna(value=x_test['Age'].mean(), inplace=True)
x_test["Fare"].fillna(value=x_test['Fare'].mean(), inplace=True)

In [7]:
print(f"{x_train.isnull().sum()}\n\n{x_test.isnull().sum()}")

PassengerId    0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

PassengerId    0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64


In [8]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train,test_size=0.2, random_state=30)

In [9]:
print(len(X_train), len(X_val), len(Y_train), len(Y_val))

712 179 712 179


In [10]:
from sklearn.linear_model import LinearRegression
regression = LinearRegression()
train_output = regression.fit(X_train, Y_train)

In [11]:
score = cross_val_score(regression, X_train, Y_train, cv=5)
print(score)

[0.39321064 0.26508677 0.32771364 0.35455526 0.36267949]


In [12]:
print(f"Accuracy of the Linear Regression:{score.mean()}")

Accuracy of the Linear Regression:0.3406491598456324


In [13]:
y_pred = regression.predict(x_test)

In [14]:
# y_pred
prediction_output = pd.DataFrame({'PassengerId':test_data['PassengerId'], 'Survived' : y_pred})
prediction_output['Survived'] = prediction_output['Survived'].astype(int)
prediction_output.to_csv('submission.csv', index=False)

In [15]:
print(f"The correlation between the predicted output and the Given prediction : {prediction_output['Survived'].corr(submission_data['Survived'])}")

The correlation between the predicted output and the Given prediction : 0.2274294130736713


In [16]:
submission_data['percent'] = submission_data['Survived'] == prediction_output['Survived']
print('Percent: ', (submission_data['percent'].value_counts('True')[1]))

Percent:  0.6650717703349283
